In [3]:
import pprint
from typing import List

In [4]:
class Matrix:
    def __init__(self, mat:list[list]):
        self.mat = mat
        self.size = (len(self.mat), len(self.mat[0]))

    def __str__(self):
        return "\n" + "\n".join(str(x) for x in self.mat) + "\n"
    
    def __eq__(self, b):
        return all([x in self.mat for x in b.mat ]) and self.size == b.size
    
    def __ne__(self, b):
        return not (self == b)

    def __getitem__(self, key):
        return self.mat[key[0]][key[1]]
    
    def __add__(self, b):
        if self.size != self.size:
            raise ValueError("Разная размерность матриц")
        mat = []
        for i in range(self.size[0]): 
            row = []
            for y in range(self.size[1]):
                row.append(self[(i, y)] + b[(i, y)])
            mat.append(row)
        return Matrix(mat)
    
    def __sub__(self, b):
        if self.size != self.size:
            raise ValueError("Разная размерность матриц")
        mat = []
        for i in range(self.size[0]): 
            row = []
            for y in range(self.size[1]):
                row.append(self[(i, y)] - b[(i, y)])
            mat.append(row)
        return Matrix(mat)
    def __mul__(self, b):
        if isinstance(b, (int, float)):
            mat = []
            for i in range(self.size[0]): 
                row = []
                for y in range(self.size[1]):
                    row.append(self[(i, y)] * b)
                mat.append(row)
            return Matrix(mat)
        
        elif isinstance(b, (Matrix, list)):
            if self.size[1] != b.size[0]:
                raise ValueError(f"Разная размерность матриц {self.size} и {b.size}" )
            mat = []
            for i in range(self.size[0]): 
                row = []
                for y in range(b.size[1]):
                    row.append(sum([self[(i, k)] * b[(k, y)] for k in range(self.size[1])]))
                mat.append(row)
            return Matrix(mat)
        
        else:
            raise TypeError("Можно умножать только на число (int или float)")
    def __rmul__(self, b):
        return self.__mul__(b)
    

    def __neg__(self):
        pass
        
    def transpose(self):
        a = []
        for i in range(self.size[1]): 
            row = []
            for j in range(self.size[0]):
                row.append(self.mat[j][i])
            a.append(row)
        return Matrix(a)
    
    def determinant(self) -> float:
        """
        Вычисляет определитель квадратной матрицы методом Гаусса.
        Вход: квадратная матрица A (n x n)
        Выход: определитель (float)
        Raises: ValueError, если матрица не квадратная
        """
        if self.size[0] != self.size[1]:
            raise ValueError("Матрица должна быть квадратной")

        n = self.size[1]
        det = 1.0
        matrix = self.mat  # Создаем копию, чтобы не менять исходную матрицу

        for col in range(n):
            # Поиск ненулевого элемента в текущем столбце
            pivot_row = None
            for row in range(col, n):
                if abs(matrix[row][col]) > 1e-10:
                    pivot_row = row
                    break

            if pivot_row is None:
                return 0.0  # Все элементы столбца нулевые -> det = 0

            # Перестановка строк
            if pivot_row != col:
                matrix.swap_rows(col, pivot_row)
                det *= -1  # При перестановке строк знак определителя меняется

            # Нормализация ведущей строки
            pivot = matrix[col][col]
            det *= pivot  # Умножаем определитель на ведущий элемент

            # Исключение элементов ниже ведущего
            for row in range(col + 1, n):
                factor = matrix[row][col] / pivot
                for c in range(col, n):
                    matrix[row][c] -= factor * matrix[col][c]

        return det

        
    

In [6]:
a = Matrix([[1, 2], [3, 4], [5, 6]])
b = Matrix([[0, 2], [2, 0], [2, 0]])
print( b.transpose() * a , a)

c = Matrix([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]])

print(c.determinant()) # Должен быть ноль

d = Matrix([
    [2, 3, 4],
    [5, 6, 7],
    [8, 9, 10]
])
print(c.determinant())



[16, 20]
[2, 4]
 
[1, 2]
[3, 4]
[5, 6]

0.0
0.0


In [ ]:
def gauss_solver(A: 'Matrix', b: 'Matrix') -> List['Matrix']:
    n = A.rows
    augmented = A.augment(b)  # Расширенная матрица [A|b]
    
    # Прямой ход: приведение к ступенчатому виду
    rank = 0
    pivot_cols = []
    for col in range(augmented.cols - 1):  # Исключаем последний столбец (вектор b)
        # Поиск ненулевого элемента в текущем столбце
        pivot_row = -1
        for row in range(rank, n):
            if abs(augmented[row][col]) > 1e-10:
                pivot_row = row
                break
        if pivot_row == -1:
            continue  # Все элементы столбца нулевые
        
        # Перестановка строк
        augmented.swap_rows(rank, pivot_row)
        # Нормализация ведущей строки
        pivot = augmented[rank][col]
        for c in range(col, augmented.cols):
            augmented[rank][c] /= pivot
        # Исключение элементов ниже ведущего
        for row in range(rank + 1, n):
            factor = augmented[row][col]
            for c in range(col, augmented.cols):
                augmented[row][c] -= factor * augmented[rank][c]
        pivot_cols.append(col)
        rank += 1

    # Проверка на несовместность
    for row in range(rank, n):
        if abs(augmented[row][-1]) > 1e-10:
            raise ValueError("Система несовместна")

    # Определение свободных переменных
    all_vars = list(range(A.cols))
    leading_vars = pivot_cols
    free_vars = [var for var in all_vars if var not in leading_vars]
    
    # Если нет свободных переменных: единственное решение
    if not free_vars:
        x = [0] * A.cols
        for row in reversed(range(rank)):
            x[leading_vars[row]] = augmented[row][-1]
            for col in range(leading_vars[row] + 1, A.cols):
                x[leading_vars[row]] -= augmented[row][col] * x[col]
        return [Matrix([x])]

    # Построение базисных векторов
    basis = []
    for var in free_vars:
        solution = [0] * A.cols
        solution[var] = 1  # Свободная переменная = 1
        # Вычисление главных переменных
        for row in reversed(range(rank)):
            col = leading_vars[row]
            solution[col] = augmented[row][-1]
            for c in range(col + 1, A.cols):
                solution[col] -= augmented[row][c] * solution[c]
        basis.append(Matrix([solution]))
    
    return basis
